In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [62]:
import requests
import json
import os

GITHUB_URL = "https://api.github.com/"

"""
Function to get JSON response from a URL
:params:
    url     string
:return:
            JSON
"""
def __get_json_response(url):
    headers = {'Authorization': 'token ' + os.environ['GITHUB_API_KEY']}
    response = requests.get(url, headers=headers)
    return json.loads(response.text)

"""
Function to add email to a set of emails and set a loop break flag
:params:
    email_set       set         set of all the emails for the user
    email           string      new email to be added
    max_len         integer     maximum number of emails to be extracted
:return:
    email_set       set         set of all the emails for the user
    break_flag      boolean     if max_limit is reached, break_flag is set to True
"""
def __add_email(email_set, email, max_len):
    email_set = email_set | set([email])
    break_flag = (len(email_set) >= max_len)

    return email_set, break_flag

"""
Function to get user emails using GitHub APIs
:params:
    user        string      a valid GitHub username
    max_limit   integer     maximum number of email ID to be fetched
:return:
    user_email  set         a set of all emails extracted
    message     string      if any error occurs, this holds the respective error message
"""
def __get_github_emails(user, max_limit):
    user_email = set([])
    break_flag = False
    try:
        users_profile_url = GITHUB_URL + "users/{0}".format(user)
        response = __get_json_response(users_profile_url)

        # some error encountered
        if 'message' in response:
            if response['message'] == 'Not Found':
                return u'You need to enter a valid GitHub Username'
            else:
                return response['message']

        name = response['name']
        user_name = response['name']
        company = response['company']
        # if user has a public email, add that to the set of emails
        if response['email']:
            user_email, break_flag = __add_email(user_email, response['email'], max_limit)

        if not break_flag:
            users_repository_url = GITHUB_URL + "users/{0}/repos?type=owner&sort=updated".format(user)
            response = __get_json_response(users_repository_url)

            for repo in response:
                if not repo['fork']:
                    users_repository_name = repo['full_name']
                    repos_commit_url = GITHUB_URL + "repos/{0}/commits".format(users_repository_name)
                    commit_reponse = __get_json_response(repos_commit_url)
                    
                    possible_positions = ['committer', 'author']

                    for commit in commit_reponse:
                        for i in possible_positions:
                            if commit['commit'][i]['name'] == user_name:
                                email_string = commit['commit'][i]['email']
                                if "noreply" not in email_string:
                                    user_email, break_flag = __add_email(user_email, email_string, max_limit)

                        if break_flag:
                            break

                if break_flag:
                    break

        if len(user_email) > 0:
            return user_email, name, company
        else:
            return u'No emails found', u'No first name found', u'No company found'

    except requests.exceptions.ConnectionError:
        return u'Proper internet connection not found', u'Proper internet connection not found', u'Proper internet connection not found'

"""
Function to get the emails associated to a username on GitHub
:params:
    username    string      a valid GitHub username
    num         integer     maximum number of email ID to be fetched, default 1
:return:
    response    JSON response
        success     boolean     flag to determine other key in JSON
        email       list        if 'success' is True, list of all the emails fetched
        message     string      if 'success' is False, returns the error message
"""
def get(username, num=1):
    github_email_response, github_full_name = __get_github_emails(username, num)
    
    if type(github_email_response) == set:
        response = {
            'success' : True,
            'data': [github_email_response, github_full_name]
        }
    else:
        response = {
            'success' : False,
            'message' : [github_email_response, github_full_name]
        }

    return response

In [67]:
def get_email_content(first_name, issue_url):
        return '''
<html>
<body>
<img src="https://www.google-analytics.com/collect?v=1&tid=UA-132339784-2&cid=555&t=event&ec=email_personal&ea=open&dp=%2Femail%2Fpersonal1&dt=Email%20Personal">
<span style="font-size:14px;"><span style="font-family:calibri,sans-serif;">Hi '''.decode('utf-8')+ first_name.split(" ")[0] +''',<br />
<br />
I saw you filed <a href="'''.decode('utf-8')+ issue_url+'''">this</a> issue on Github and I want to help you <strong>solve it quickly</strong>. We are a <a href="http://speedoss.com">platform</a> for <strong>tying rewards to your Github issues</strong>. These rewards go a long way in making you stand out. They are a GREAT way to:

<ul>
	<li>get your issue solved quickly</li>
	<li>financially support the open source project</li>
	<li>develop strong relationships with core contributors (for hiring the best talent)</li>
</ul>
Many businesses love our platform so much so they develop entire applications on it; simply by creating blank Github repos and filing <a href="http://speedoss.com">incentivized Github Issues</a>&nbsp;- they build entire applications at a fraction of the cost while using the best engineering talent available.<br />
<br />
Do you have a moment this week for a quick conversation? I think there's a way for us to do some business together and perhaps provide significant software engineering horsepower to you and your team.<br />
<br />
Let me know a good time for me to call you.<br />
<br />
Best Regards,<br />
Charlie<br />
Speed OSS Inc. |&nbsp;<a href="http://speedoss.com">speedoss.com</a></span></span>
</body>
</html>
        '''.decode('utf-8')
    

In [68]:
from __future__ import print_function
from googleapiclient.discovery import build
from apiclient import errors
from httplib2 import Http
from email.mime.text import MIMEText
import base64
from google.oauth2 import service_account
import logging

logging.getLogger('googleapicliet.discovery_cache').setLevel(logging.ERROR)

def create_message(sender, to, subject, message_text):
  """Create a message for an email.
  Args:
    sender: Email address of the sender.
    to: Email address of the receiver.
    subject: The subject of the email message.
    message_text: The text of the email message.
  Returns:
    An object containing a base64url encoded email object.
  """
  message = MIMEText(message_text, 'html')
  message['to'] = to
  message['from'] = sender
  message['subject'] = subject
  return {'raw': base64.urlsafe_b64encode(message.as_string())}

def send_message(service, user_id, message):
  """Send an email message.
  Args:
    service: Authorized Gmail API service instance.
    user_id: User's email address. The special value "me"
    can be used to indicate the authenticated user.
    message: Message to be sent.
  Returns:
    Sent Message.
  """
  try:
    message = (service.users().messages().send(userId=user_id, body=message)
               .execute())
    print('Message Id: %s' % message['id'])
    return message
  except errors.HttpError as error:
    print('An error occurred: %s' % error)

def service_account_login(EMAIL_FROM):
  SCOPES = ['https://www.googleapis.com/auth/gmail.send']
  SERVICE_ACCOUNT_FILE = 'Speed-d90429ee3804.json'

  credentials = service_account.Credentials.from_service_account_file(
          SERVICE_ACCOUNT_FILE, scopes=SCOPES)
  delegated_credentials = credentials.with_subject(EMAIL_FROM)
  service = build('gmail', 'v1', credentials=delegated_credentials)
  return service




In [69]:
emails = {}
def send_github_proactive_email(USER_ID, ISSUE_URL, REPO_NAME, IS_LIVE, IS_PERSONAL):
    resp = get(USER_ID)
    if 'data' not in resp:
        return 'NO EMAIL FOUND for user: {}'.format(USER_ID)
    EMAIL_TO, FIRST_NAME = get(USER_ID)['data']
    EMAIL_TO = EMAIL_TO.pop()
    print("FOUND EMAIL {} FOR USER {}".format(EMAIL_TO, USER_ID))
    if not IS_LIVE:
        EMAIL_TO = 'daljeetv@gmail.com'
    EMAIL_CONTENT = get_email_content(FIRST_NAME, ISSUE_URL, IS_PERSONAL)
    EMAIL_FROM = 'charlie@speedoss.com'
    EMAIL_SUBJECT = 'Regarding your open issue'
    service = service_account_login(EMAIL_FROM)
    # Call the Gmail API
    if EMAIL_TO not in emails:
        message = create_message(EMAIL_FROM, EMAIL_TO, EMAIL_SUBJECT, EMAIL_CONTENT)
        sent = send_message(service,'me', message)
        emails[EMAIL_TO] = True
        return 'SENT EMAIL TO {}'.format(EMAIL_TO)
    else:
        return 'REPEAT EMAIL!'

In [61]:
import pandas as pd
df = pd.read_csv('results-20190114-145147.csv')
print(df.columns)
row_iterator = df.iterrows()
start_at = 1001
end_at = 1002
for x in range(0, end_at+1):
    index, row = row_iterator.next()
    html_url   = row['html_url']
    repo_name  = row['name']
    url        = row['url']
    login      = row['login']
    created_at = row['created_at']
    stars      = row['stars']
    # Email variables. Modify this!
    if x >= start_at and x < end_at:
        try:
            USER_ID=login
            ISSUE_URL=html_url
            REPO_NAME=repo_name
            IS_LIVE=False
            EMAIL_TYPES={'ENTERPRISE', 'MARKETING', 'PERSONAL'}
            TYPE_OF_EMAIL= 'ENTERPRISE'
            sent_to = send_github_proactive_email(USER_ID, ISSUE_URL, REPO_NAME, IS_LIVE, TYPE_OF_EMAIL)
            print(x, sent_to)
        except Exception as e:
            print("UNABLE TO SEND EMAIL: {}".format(e))

Index([u'html_url', u'html_url2', u'name', u'url', u'login', u'created_at',
       u'stars', u'f0_'],
      dtype='object')
{u'public_repos': 8, u'site_admin': False, u'subscriptions_url': u'https://api.github.com/users/georgesterpu/subscriptions', u'gravatar_id': u'', u'hireable': None, u'id': 6018251, u'followers_url': u'https://api.github.com/users/georgesterpu/followers', u'following_url': u'https://api.github.com/users/georgesterpu/following{/other_user}', u'blog': u'https://georgesterpu.github.io/', u'followers': 7, u'location': u'Ireland', u'type': u'User', u'email': None, u'bio': None, u'gists_url': u'https://api.github.com/users/georgesterpu/gists{/gist_id}', u'company': u'Trinity College Dublin', u'events_url': u'https://api.github.com/users/georgesterpu/events{/privacy}', u'html_url': u'https://github.com/georgesterpu', u'updated_at': u'2019-01-13T15:30:43Z', u'node_id': u'MDQ6VXNlcjYwMTgyNTE=', u'received_events_url': u'https://api.github.com/users/georgesterpu/received_eve

KeyboardInterrupt: 